In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import math
import absl
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#data = input_data.read_data_sets('data/MNIST/', one_hot=True)
mnist = input_data.read_data_sets('./mnist', one_hot=True)  # they has been normalized to range (0,1)
test_x = mnist.test.images[:10000]
test_y = mnist.test.labels[:10000]

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.test.cls = np.argmax(mnist.test.labels, axis=1)

In [4]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x') / 255
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [5]:
np.random.shuffle(mnist.train.labels)

In [6]:
#network1
net = tf.layers.conv2d(inputs=x_image, name='layer_conv1_1', padding='same',
                       filters=16, kernel_size=5, activation=tf.nn.relu)
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

# layer_conv2
net = tf.layers.conv2d(inputs=net, name='layer_conv2_1', padding='same',
                       filters=36, kernel_size=5, activation=tf.nn.relu)
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

print(net)
net = tf.layers.flatten(net)

print(net)
net = tf.layers.dense(inputs=net, name='layer_fc1_1',
                      units=128, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=net, name='layer_fc_out_1',
                      units=num_classes, activation=None)
print(logits)
y_pred = tf.nn.softmax(logits=logits)
y_pred_cls = tf.argmax(y_pred, dimension=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits)
loss = tf.reduce_mean(cross_entropy)

Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 7, 7, 36), dtype=float32)
Tensor("flatten/Reshape:0", shape=(?, 1764), dtype=float32)
Tensor("layer_fc_out_1/BiasAdd:0", shape=(?, 10), dtype=float32)


/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core

In [7]:
def get_weights_variable(layer_name):
    
    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('kernel')

    return variable

total_weights = []
weights_conv1 = get_weights_variable(layer_name='layer_fc_out_1')

print(weights_conv1)

<tf.Variable 'layer_fc_out_1/kernel:0' shape=(128, 10) dtype=float32_ref>


In [11]:
opt = tf.train.AdamOptimizer(learning_rate=1e-4)
optimizer = opt.minimize(loss)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
train_batch_size = 64
loss_list=[]
loss_list_test=[]
accuracy_list=[]
accuracy_list_test=[]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    for iteration in range(200000):
        x_batch, y_true_batch = mnist.train.next_batch(train_batch_size)
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        sess.run(optimizer, feed_dict = feed_dict_train)
        if iteration % 100 == 0:
            feed_dict_test = {x: mnist.test.images, y_true: mnist.test.labels}
            y_true_cls_batch = np.argmax(y_true_batch, axis=1) 
            loss_,accuracy_ = sess.run([loss,  accuracy], feed_dict = feed_dict_train)
            loss_test_, accuracy_test_ = sess.run([loss, accuracy], feed_dict = feed_dict_test)
            loss_list.append(loss_)
            loss_list_test.append(loss_test_)
            accuracy_list.append(accuracy_)
            accuracy_list_test.append(accuracy_test_)

KeyboardInterrupt: 